## Tapis PEARC20 Demo

In this notebook, we use Tapis to store and analyze streaming data generated from code simulating a sensor. We introduce a number of Tapis services and concepts along the way.

![Alt text](images/Tapisworkflow.png "a title")

### Tapis Python SDK, Tenants and Authentication

In this notebook, we will use the official Tapis Python SDK for all of our interactions with the services. The Python SDK provides Python-native methods and objects for making HTTP requests and parsing HTTP responses to and from the Tapis API. 

In order to do just about anything with Tapis, we will need to authenticate. Tapis makes heavy use of the notion of "tenants" in order to provide isolation for different projects. By setting the base_url variable, you indicate to the Tapis SDK which tenant you wish to interact with.

For the demo, we will be using a base_url of "https://tacc.tapis.io" -- the "TACC tenant" -- which will allow us to authenticate using any valid TACC account. For other tenants, the authentication rules could be different. 

Authentication in the TACC tenant uses OAuth2 (again, this could be different in other tenants), but the Tapis Python SDK simplifies some of the complexity inherent in OAuth2 by providing some convenience functions for common use cases. For example, we are able to generate an access token using just our username and password via the convenience function “get_tokens()”. We do this below:

In [ ]:
import datetime
import getpass
import os
#Setup Variables that are used in the rest of the notebook
permitted_username = getpass.getpass(prompt='Permitted Username: ', stream=None)
permitted_user_password = getpass.getpass(prompt='Permitted Password: ', stream=None)
non_permitted_username = getpass.getpass(prompt='Non Permitted Username: ', stream=None)
non_permitted_password = getpass.getpass(prompt='Non Permitted Password: ', stream=None)
tenant="dev"
base_url = 'https://dev.tapis.io'

#update project id so the rest can create in as unique objects
project_id ='wq_demo_tapis_proj1'+ str(datetime.datetime.today().isoformat())

site_id = 'wq_demo_site'
instrument_id = 'Ohio_River_Robert_C_Byrd_Locks'
channel_id = 'demo_wq_channel'+  str(datetime.datetime.today().isoformat()).replace(':','_')
template_id = 'demo_channel_template'
nonce = os.environ['ABACO_NONCE']
actor_id = "Aw1ebg4GG1JrO"
storage_id = "tapis-demo"

In [ ]:
#Load Python SDK
from tapy.dyna import DynaTapy 

#Create python Tapis client for user
permitted_client = DynaTapy(base_url= base_url, username=permitted_username, password=permitted_user_password, account_type='user', tenant_id='dev') 
permitted_client.get_tokens()

#Create python Tapis client for non-permitted user
nonpermitted_client = DynaTapy(base_url= base_url, username=non_permitted_username, password=non_permitted_password, account_type='user', tenant_id='dev') 
nonpermitted_client.get_tokens()

In Tapis, access tokens (and refresh tokens) are simply JSON Web Tokens (JWTs). The access_token Python object created and managed by the Python SDK has attributes on it that include the "raw" JWT string as well as claims associated with the JWT. Services use the claims to determine what actions a user is authorized to take. In particular, the "sub" (subject) claim uniquely identifies a user inside Tapis. 

In [ ]:
permitted_client.access_token

Note also the ttl (time-to-live) claim; Tapis tokens have a finite lifetime, typically a few hours, configurable by tenant. After the token expires, we will need to get a new token in order to continue interacting with Tapis. The Python SDK has convenience methods for managing tokens and even automatically refreshing a token.

## Systems API

In [ ]:
# Look at the system description for our pre-registered S3 bucket:

permitted_client.systems.getSystemByName(systemName=storage_id)

In [ ]:
# Code used to actually create the storage system -- we will not run here.

# the description of an S3 bucket
s3_bucket = {
  "name":storage_id,
  "description":"Joe's Bucket",
  "host":"https://tapis-files-test.s3.us-east-1.amazonaws.com/",
  "systemType":"OBJECT_STORE",
  "defaultAccessMethod":"ACCESS_KEY",
  "effectiveUserId":"testuser2",
  "bucketName":"tapis-files-bucket",
  "rootDir":"/",
  "jobCanExec": False,
  "transferMethods":["S3"],
  "accessCredential":
  {
    "accessKey":"***",
    "accessSecret":"***"
  }
}

# create the system in Tapis
# permited_client.systems.createSystem(**s3_bucket)


## Files API

In [ ]:
#List file of current storage system
permitted_client.files.listFiles(systemId=storage_id, path="/")

## Streams API

![Alt text](images/streams-api.png "a title")

### Project and Metadata Setup
Projects are defined at a top level in the hierarchy of Streams resources. A user registers a project by providing metadata information such as the principal Investigator, project URL, funding resource, etc. A list of authorized users can be added to various project roles to have a controlled access over the project resources. When a project is first registered, a collection is created in the back-end MongoDB. User permissions to access this collection are then set up in the security kernel. Every request to access the project resource or documents within (i.e sites, instruments, variables) goes through a security kernel check and only the authorized user requests are allowed to be processed.

In [ ]:
## Create Project
result, debug = permitted_client.streams.create_project(project_name=project_id,description='project for early adopters demo',
                                         owner='testuser2', pi='ajamthe', funding_resource='tapis', project_url='test.tacc.utexas.edu',
                                         active=True,_tapis_debug=True)
print(result)

![Alt text](images/stream-mongo.png "a title")

#### Create Site
A site is a geographical location that may hold one or more instruments. Sites are next in the streams hierarchy and they inherit permissions from the projects. Project owners can create sites by providing the geographical information such as latitude, longitude and elevation of the site or GeoJSON encoded spatial information. This spatial information is useful when searching sites or data based on location. In the back-end database a site is represented as a JSON document within the project collection. Site permissions are inherited from the project. 


In [ ]:
## Create Site
result, debug = permitted_client.streams.create_site(project_uuid=project_id,site_name=site_id, site_id=site_id,
                                      latitude=50, longitude = 10, elevation=2,description='test_site', _tapis_debug=True)
print(result)

#### Create Instrument
Instruments are physical entities that may have one or more embedded sensors to sense various parameters such as temperature, relative humidity, specific conductivity, etc. These sensors referred to as variables in Streams API generate measurements, which are stored in the influxDB along with a ISO8601 timestamp. Instruments are associated with specific sites and projects. Information about the instruments such as site and project ids, name and description of the instrument, etc. are stored in the mongoDB sites JSON document. 

In [ ]:
## Create Instruments
result, debug = permitted_client.streams.create_instrument(project_uuid=project_id,topic_category_id ='2',site_id=site_id,
                                            inst_name=instrument_id,inst_description='demo instrument',
                                            inst_id=instrument_id, _tapis_debug=True)
print(result)

#### Create Variables
Variables are associated with specific instruments. When a variable is created the users provide information such as the name of variable, properties measured, units of measurements, etc. For example, a variable for temperature sensor when created can store measurements in degree Celsius or Fahrenheit.

In [ ]:
## Create Variables 
#Temperature
result, debug = permitted_client.streams.create_variable(project_uuid=project_id,topic_category_id ='2',
                                          site_id=site_id,inst_id=instrument_id,
                                          var_name='temperature', shortname='temp',var_id='temp', _tapis_debug=True)
print(result)
#Battery Voltage
result, debug = permitted_client.streams.create_variable(project_uuid=project_id,topic_category_id ='2',
                                          site_id=site_id,inst_id=instrument_id,
                                          var_name='battery', shortname='bat', var_id='batv',
                                          _tapis_debug=True)
print(result)
#Specific Conductivity
result, debug = permitted_client.streams.create_variable(project_uuid=project_id,topic_category_id ='2',
                                          site_id=site_id,inst_id=instrument_id,
                                          var_name='specific_conductivity', shortname='spc', var_id='spc',
                                          _tapis_debug=True)
print(result)
#Turbidity
result, debug = permitted_client.streams.create_variable(project_uuid=project_id,topic_category_id ='2',
                                          site_id=site_id,inst_id=instrument_id,
                                          var_name='turbidity', shortname='turb', var_id='turb',
                                          _tapis_debug=True)
print(result)
#PH
result, debug = permitted_client.streams.create_variable(project_uuid=project_id,topic_category_id ='2',
                                          site_id=site_id,inst_id=instrument_id,
                                          var_name='ph_level', shortname='ph', var_id='ph',
                                          _tapis_debug=True)
print(result)

### Stream Permissions
Project roles and permissions are stored in the Tapis v3 Security Kernal.

In [ ]:
#Project Roles
permitted_client.sk.getUsersWithRole(user=permitted_username, roleName='streams_user',tenant=tenant)


In [ ]:
nonpermitted_client.sk.getUserRoles(user=non_permitted_username,tenant=tenant)

In [ ]:
nonpermitted_client.streams.list_projects()

In [ ]:
## Site Role  - non-permissioned user
result = nonpermitted_client.streams.get_site(project_uuid=project_id, site_id=site_id)
print(result)

In [ ]:
## Site Role - permitted user
result = permitted_client.streams.get_site(project_uuid=project_id, site_id=site_id)
print(result)

### Write Measurements
Measurements are actual values from the variables, which are stored in the time series database influxDB. Project owners or users can download these measurements by providing a time window of measurement creation and retrieve the data in the CSV or JSON format. This data  can be processed in real time with the help of the Channels API.

In [ ]:
#Write Measurements
from datetime import datetime
import random
from random import randint
for i in range(0, 10):
    datetime_now = datetime.now().isoformat()
    result = permitted_client.streams.create_measurement(inst_id=instrument_id,
                                          vars=[{"var_id": "temp", "value": randint(85, 89)},
                                                {"var_id": "spc", "value": randint(240, 300)},
                                                {"var_id": "turb", "value": randint(10, 19)},
                                                {"var_id": "ph", "value": randint(1, 10)},
                                                {"var_id": "batv", "value": round(random.uniform(10, 13), 2)}],
                                          datetime=datetime_now)
    print(result)

### Download Measurements
Download the measurements we just created from our instrument.

In [ ]:
#Download measurments as CSV
result = permitted_client.streams.list_measurements(inst_id=instrument_id,project_uuid=project_id, site_id=site_id,
                                             start_date='2020-05-08T00:00:00Z',end_date='2020-12-30T22:19:25Z',format='csv')
result

In [ ]:
#Read Measurements to Data Frame
import pandas as pd
from io import StringIO
input = StringIO(str(result,'utf-8'))
df = pd.read_csv(input)
df['datetime']=pd.to_datetime(df['time'])
df.set_index('datetime',inplace=True)
df.pop('time')
df

In [ ]:
# Plot Measurements in the DataFrame
import matplotlib.pyplot as plt
import matplotlib.dates as md
%matplotlib inline
xfmt = md.DateFormatter('%H:%M:%S')
df.plot(lw=1, colormap='jet', marker='.', 
        markersize=12, title='Timeseries Stream Output', rot=90).xaxis.set_major_formatter(xfmt)
plt.tight_layout()
plt.legend(loc='best')
plt.savefig('test2.png')

### Create Actor

We create an Abaco actor to automatically execute code whenever a sensor registers a temperature reading beyond a predefined temperature threshold, in this case, 90 degrees. The actor is a standalone function that is written in Python and packaged as a Docker container.

The code for the actor is available from within the actor directory. When executed, the actor code retrieves all stream data points for the given instrument. It then makes a plot of the data points and uploads the plot to a Tapis S3 bucket.


In [ ]:
# Review actor code

### Create Channel
Channels are created using the variable resources. When the data for a specific variable meets a certain condition defined in the channel, an alert or notification is raised. The Channels API leverages Kapacitor to create tasks to process real time streaming data from influxDB.

##### Channels require a "template" we will use an existing one 'demo_channel_template' that accepts a single condition

In [ ]:
##Create Channel
result,debug =  permitted_client.streams.create_channels(channel_id=channel_id, channel_name='demo.wq.channel', template_id=template_id,
                                              triggers_with_actions=[{"inst_ids":["Ohio_River_Robert_C_Byrd_Locks"],
                                                                      "condition":{"key":"Ohio_River_Robert_C_Byrd_Locks.temp","operator":">", "val":90}, 
                                                                      "action":{"method":"ACTOR","actor_id" :actor_id,
                                                                                "message":"Instrument: Ohio_River_Robert_C_Byrd_Locks temp exceeded threshold", 
                                                                                "abaco_base_url":"https://api.tacc.utexas.edu","nonces": nonce}}]
                                              ,_tapis_debug=True)
print(result)

## Alerts
As events are generated an "Alert" is created, notifications are sent to the data processing end-points via HTTP POST with details ofthe data raising the alerts

![Alt text](images/alert-abaco.png "a title")

In [ ]:
# list empty Alerts
permitted_client.streams.list_alerts(channel_id=channel_id)

In [ ]:
# Trigger Alert Measurement
from datetime import datetime
import random
from random import randint
datetime_now = datetime.now().isoformat()
result = permitted_client.streams.create_measurement(inst_id=instrument_id,
                                      vars=[{"var_id": "temp", "value": 150},
                                            {"var_id": "spc", "value": randint(240, 300)},
                                            {"var_id": "turb", "value": randint(10, 19)},
                                            {"var_id": "ph", "value": randint(1, 10)},
                                            {"var_id": "batv", "value": round(random.uniform(10, 13), 2)}],
                                      datetime=datetime_now)
print(result)
for i in range(0, 5):
    datetime_now = datetime.now().isoformat()
    result = permitted_client.streams.create_measurement(inst_id=instrument_id,
                                          vars=[{"var_id": "temp", "value": randint(85, 89)},
                                                {"var_id": "spc", "value": randint(240, 300)},
                                                {"var_id": "turb", "value": randint(10, 19)},
                                                {"var_id": "ph", "value": randint(1, 10)},
                                                {"var_id": "batv", "value": round(random.uniform(10, 13), 2)}],
                                          datetime=datetime_now)
    print(result)

### List Alerts

In [ ]:
#list alerts after trigger
permitted_client.streams.list_alerts(channel_id=channel_id)

In [ ]:
permitted_client.files.listFiles(systemId=storage_id, path="/")

### Angular UI Filebrowser
Go to the file browser to preview the generated plot https://tapis-project.github.io/ng-tapis-files-browser